In [16]:
import cv2
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 1️⃣ 데이터 경로 설정
IMG_FILE = '../project9/DATA/normal_cup/desert39 (43).png'
TARGET_FILE = '../project9/DATA/normal_cup/desert39 (44).png'

STANLEY_FILE = '../project9/DATA/normal_cup_st/stanley (169).png'
STANLEY_TARGET_FILE = '../project9/DATA/normal_cup_st/stanley (170).png'

IMG_SIZE = (224, 224)  # 이미지 크기 통일
THRESHOLD = 0.97  # 상관계수 기준 (0.95보다 낮으면 재분류)

# 2️⃣ 이미지 로드 및 크기 조정
def load_and_preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, IMG_SIZE)  # 크기 조정
    return img.flatten()  # 1차원 배열로 변환

# 3️⃣ 데이터 저장 리스트
data = []
labels = []

# 4️⃣ Desert39 데이터 생성
desert39_img = load_and_preprocess_image(IMG_FILE)
desert39_target = load_and_preprocess_image(TARGET_FILE)
data.append(desert39_img)
labels.append("Desert39")

# 노이즈 이미지 200장 생성
for i in range(200):
    noise = np.random.normal(0, i / 20, desert39_img.shape).astype(np.uint8)
    noisy_img = np.clip(desert39_img + noise, 0, 255)
    data.append(noisy_img)
    labels.append("Desert39")

# 5️⃣ Stanley 데이터 생성
stanley_img = load_and_preprocess_image(STANLEY_FILE)
stanley_target = load_and_preprocess_image(STANLEY_TARGET_FILE)
data.append(stanley_img)
labels.append("Stanley")

for i in range(200):
    noise = np.random.normal(0, i / 20, stanley_img.shape).astype(np.uint8)
    noisy_img = np.clip(stanley_img + noise, 0, 255)
    data.append(noisy_img)
    labels.append("Stanley")

# 6️⃣ 데이터프레임 생성 및 저장
df = pd.DataFrame(data)
df["Label"] = labels
df.to_csv("../project9/noisy_cup_data.csv", index=False)

print(f"✅ Desert39 & Stanley 데이터 400장 생성 완료!")

# 7️⃣ Desert39(43) vs Desert39(44) 상관계수 계산 및 검증
corr_coef = np.corrcoef(desert39_img, desert39_target)[0, 1]
print(f"📌 Desert39(43) vs Desert39(44) 상관계수: {corr_coef:.4f}")

# 8️⃣ 머신러닝 모델 학습 (SVM 사용)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

model = SVC(kernel="linear", probability=True)  # 확률 예측 가능하도록 설정
model.fit(X_train, y_train)

# 9️⃣ 테스트 및 정확도 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ 머신러닝 모델 정확도: {accuracy:.4f}")

# 🔟 새로운 이미지 분류 함수 (재분류 조건 강화)
def classify_image(img_path):
    img = load_and_preprocess_image(img_path)
    proba = model.predict_proba([img])[0]  # 각 클래스에 대한 확률
    max_proba = np.max(proba)  # 가장 높은 확률

    # 확률이 80% 미만이면 재분류 필요
    if max_proba < 0.8:
        return "⚠️ 재분류 필요"
    
    prediction = model.predict([img])[0]
    
    # Desert39 또는 Stanley가 아닐 경우 재분류 요청
    if prediction not in ["Desert39", "Stanley"]:
        return "⚠️ 재분류 필요"
    
    return prediction

# 테스트 이미지 분류
test_img = "../project9/DATA/normal_cup_st/stanley (207).png"
if os.path.exists(test_img):
    # 1️⃣ Desert39(43) vs Desert39(44) 상관계수 계산
    corr_coef = np.corrcoef(desert39_img, desert39_target)[0, 1]
    print(f"📌 Desert39(43) vs Desert39(44) 상관계수: {corr_coef:.4f}")
    
    # 2️⃣ 상관계수가 기준 이상일 경우에만 결과 출력
    if corr_coef >= THRESHOLD:
        result = classify_image(test_img)
        print(f"📌 테스트 이미지 분류 결과: {result}")
    
    # 3️⃣ 상관계수가 기준 미만일 경우에도 재분류 메시지 출력
    if corr_coef < THRESHOLD:
        print("⚠️ 일치도가 낮음 → 재분류(폐기 or 재프린팅/판매) 필요")

✅ Desert39 & Stanley 데이터 400장 생성 완료!
📌 Desert39(43) vs Desert39(44) 상관계수: 0.9783
✅ 머신러닝 모델 정확도: 1.0000
📌 Desert39(43) vs Desert39(44) 상관계수: 0.9783
📌 테스트 이미지 분류 결과: Stanley


이미지끼리의 상관계수의 정도를 뽑아서 특정 정답 데이터와 상관관계가 설정값 이상이면 일치한다!
그런데 그 정도 기준이 안 되고 재분류가 나오는 경우에는 로고가 지워져서일수도 아예 다른 브랜드일 수도 있기 때문에 재분류 카테고리로 보낸다! 

In [17]:
import joblib

# 🔹 1️⃣ SVM 모델 저장
joblib.dump(model, "../project9/cgi-bin/svm_model.pkl")
print("✅ 모델이 'svm_model.pkl'로 저장됨!")

# # 🔹 2️⃣ 저장된 모델 불러오기
# loaded_model = joblib.load("../project9/svm_model.pkl")

# # 🔹 3️⃣ 불러온 모델로 예측
# result = loaded_model.predict([load_and_preprocess_image(test_img)])
# print(f"📌 불러온 모델의 테스트 이미지 분류 결과: {result[0]}")

✅ 모델이 'svm_model.pkl'로 저장됨!


In [18]:
# import cv2
# import numpy as np
# import os
# import pandas as pd
# import joblib
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score

# # 1️⃣ 데이터 경로 설정
# IMG_SIZE = (224, 224)  # 이미지 크기 통일
# THRESHOLD = 0.97  # 상관계수 기준 (0.95보다 낮으면 재분류)

# def load_and_preprocess_image(img_path):
#     """이미지를 불러와서 그레이스케일 변환 및 크기 조정 후 1차원 배열로 변환"""
#     if not os.path.exists(img_path):
#         raise FileNotFoundError(f"파일을 찾을 수 없습니다: {img_path}")
#     img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#     img = cv2.resize(img, IMG_SIZE)
#     return img.flatten()

# # 🔟 새로운 이미지 분류 함수 (재분류 조건 강화)
# def classify_image(img_path, model):
#     try:
#         img = load_and_preprocess_image(img_path)
#         proba = model.predict_proba([img])[0]  # 각 클래스에 대한 확률
#         max_proba = np.max(proba)  # 가장 높은 확률

#         if max_proba < 0.8:
#             return "⚠️ 재분류 필요"
        
#         prediction = model.predict([img])[0]
        
#         if prediction not in ["Desert39", "Stanley"]:
#             return "⚠️ 재분류 필요"
        
#         return prediction
#     except FileNotFoundError as e:
#         return str(e)

# # 🔹 저장된 SVM 모델 불러오기
# model_path = "../project9/cgi-bin/svm_model.pkl"
# if os.path.exists(model_path):
#     model = joblib.load(model_path)
#     print("✅ 모델이 성공적으로 로드됨!")
# else:
#     raise FileNotFoundError("❌ SVM 모델 파일을 찾을 수 없습니다.")

# # 🔹 사용자 입력을 통한 이미지 분류 실행
# while True:
#     test_img = input("🔹 테스트할 이미지 경로를 입력하세요 (종료하려면 'exit' 입력): ")
#     if test_img.lower() == 'exit':
#         print("🔹 프로그램을 종료합니다.")
#         break
    
#     result = classify_image(test_img, model)
#     print(f"📌 테스트 이미지 분류 결과: {result}")
